# PROGETTO SOCIAL MEDIA MINING A.A. 2024/25 UNIMI
## Reddit Loves Rolex?

**NB:** Ho rimosso tutti gli output dal notebook. Ora il file è completamente pulito e leggero, pronto per essere visualizzato su GitHub.

# Connessione alle API di Reddit

Nel blocco di seguito ho configuarato la connessione a Reddit utilizzando **"PRAW"** (Python Reddit API Wrapper), la libreria che mi permette di interagire  con i contenuti pubblici di Reddit come post, commenti, utenti, subreddit, ecc...

Prima di tutto ho cercato di stabilire una connessione con l'API, testando che tutto funzioni correttamente, stampando un post dal subreddit **"r/rolex"**.

**Come ho ottenuto le API Key di Reddit?**

1. Ho creato un account su Reddit: https://www.reddit.com/
2. Sono andato su: https://www.reddit.com/prefs/apps
3. Ho cliccato su “create app” e selezionato il tipo "script"
4. Ho inserito:
   - Nome: "Inserisci NOME"
   - Redirect URI: "http://localhost"
5. Dopo la creazione ho ottenuto:
   - "client_id"
   - "client_secret"
   - "user_agent"

Con queste credenziali ho l'accesso ai contenuti pubblici senza l'obbligo di effettuare  il login con username e password.

In [ ]:
import praw

client_id = ''
client_secret = ''
user_agent = ' '


reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)


for post in reddit.subreddit('rolex').hot(limit=1):
    print(f"TITOLO: {post.title}")


Dalla stampa del codice si nota che ho configurato nel modo corretto le API di Reddit stabilendo una connessione via "praw".

Ho recuperato e stampato un post reale dal subreddit "r/rolex" dal titolo:
"Watch Verification Thread – If you're uncertain if a Rolex is good/bad/fake…"

Il mio prossimo obiettivo e cercare e raccogliere commenti che contengono **"gmt", "submariner", "datejust",** ovvero modelli inconici della casa orologeria di Rolex.

# Estrazione dei commenti Reddit contenenti i modelli Rolex (GMT, Submariner, DateJust)

Nel prossimo blocco di codice eseguo una raccolta mirata di commenti nei subreddit **"r/rolex" e "r/watches"**.  
L'obiettivo è identificare e raccogliere solo quei commenti che contengono riferimenti espliciti ai tre modelli di Rolex oggetto della mia analisi:

1. Cerco post recenti contenenti la parola "Rolex" in "r/rolex" e "r/watches".
2. Per ciascun post analizzo tutti i commenti pubblici e filtro quelli che contengono almeno una delle parole chiave dei modelli GMT, Submariner e DateJust.
3. Per ogni commento valido salvo:
   - Autore del commento
   - Modello menzionato (es. "submariner")
   - Testo completo del commento
   - Timestamp della pubblicazione (convertito in formato leggibile)
4. Salvo tutto in un file CSV (**"reddit_comments_rolex.csv"**) per l'uso successivo per la costruzione del grafo per Gephi e l'analisi della mia rete.
5. Inoltre ho messo un ritardo tra le richieste per evitate limiti di rate da parte di Reddit.

In [ ]:
import pandas as pd
import time

subreddits = ['rolex', 'watches']
keywords = ['gmt', 'submariner', 'datejust']
search_queries = ['Rolex GMT', 'Rolex Submariner', 'Rolex DateJust', 'Rolex GMT-Master II', 'Rolex Hulk', 'Rolex Batman', 'Rolex Pepsi']
limit_per_query = 30000  

data = []
seen_ids = set()  # Per evitare duplicati

for sub in subreddits:
    print(f"🔍 Subreddit: r/{sub}")
    for query in search_queries:
        print(f"   → Query: {query}")
        for submission in reddit.subreddit(sub).search(query, sort="new", limit=limit_per_query):
            if submission.id in seen_ids:
                continue
            seen_ids.add(submission.id)

            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                if comment.body and comment.author:
                    text = comment.body.lower()
                    for model in keywords:
                        if model in text:
                            data.append({
                                'author': comment.author.name,
                                'model': model,
                                'text': text,
                                'created_utc': pd.to_datetime(comment.created_utc, unit='s')
                            })
            time.sleep(1)

df = pd.DataFrame(data)
df.to_csv('reddit_comments_rolex.csv', index=False)
print(f"Completato: Commenti salvati: {len(df)}")
df.head()


In questo modo ho cercato su:
- **subreddits**: in quale community cercare i post Reddit (ha uno "scope" ristretto su Rolex e Watches)
- **keywords**: cosa cercare nei commenti (il testo nei commenti dopo che li ha scaricati)
- **search_queries**: cosa cercare nei post per trovarli più facilmente (sia il titolo che dentro ai post)
Ed ho salvato su un file .csv tutti i commenti raccolti, ovvero 5265

Adesso eseguo un **salvataggio dei dizionari in formato JSON,** così evito di dover rieseguire lo scraping da Reddit ad ogni esecuzione del notebook.
I dati estratti vengono salvati in locale sotto forma di dizionari JSON.

In [ ]:
import json

for d in data:
    d['created_utc'] = d['created_utc'].isoformat()

with open('reddit_comments_data.json', 'w') as comments_file:
    json.dump(data, comments_file, indent=4)

with open('seen_ids.json', 'w') as seen_file:
    json.dump(list(seen_ids), seen_file, indent=4)

print("Dizionari salvati correttamente in file JSON.")


"**reddit_data.json**" contiene tutti i commenti estratti, con autore, testo, modello menzionato e timestamp.
"**seen_ids.json**" contiene l’elenco degli ID dei post già processati, utile per evitare duplicati nelle future esecuzioni.

# Analisi dei commenti per modello e subreddit
Grazie al successivo codice conto il numero di commenti raccolti per ciascun modello di Rolex (gmt, submariner e datejust) in base al testo.
Distinguo i subreddit da cui provengono i commenti da r/rolex o r/watches.
Visualizzo i risultati in:
- Una tabella testuale riepilogativa
- Un grafico a barre che mostra la distribuzione dei commenti per modello e subreddit

In [ ]:
from collections import defaultdict

# Conto i commenti per modello e subreddit
model_counts = defaultdict(lambda: defaultdict(int))
for _, row in df.iterrows():
    subreddit = 'rolex' if 'rolex' in row['text'] else 'watches'
    model_counts[subreddit][row['model']] += 1

print("\nRiepilogo commenti per modello e subreddit:")
for sub, models in model_counts.items():
    for model, count in models.items():
        print(f"r/{sub:<8} → {model:<12}: {count} commenti")


In [ ]:
import matplotlib.pyplot as plt

# Preparo i dati per il grafico
data_for_plot = []
for subreddit, models in model_counts.items():
    for model, count in models.items():
        data_for_plot.append({'Subreddit': subreddit, 'Modello': model.capitalize(), 'Commenti': count})

df_counts = pd.DataFrame(data_for_plot)

plt.figure(figsize=(10, 6))
for subreddit in df_counts['Subreddit'].unique():
    subset = df_counts[df_counts['Subreddit'] == subreddit]
    plt.bar(subset['Modello'] + f' ({subreddit})', subset['Commenti'], label=f"r/{subreddit}")

plt.title("Numero di commenti per modello e subreddit")
plt.xlabel("Modello Rolex (per subreddit)")
plt.ylabel("Numero di commenti")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.grid(axis='y')
plt.show()


# Analisi con Google Trends

**Analisi temporale dell'interesse per i modelli Rolex tramite Google Trends (2013–2025)**
In questa sezione del progetto, voglio analizzare l'evoluzione dell'interesse pubblico (sul prezzo) per i tre di Rolex con un'ottica temporale di lungo periodo dal 2013 al 2025.

**Obiettivo:**
Voglio identificare i periodi caldi, in hype, dove  l'interesse online per questi modelli ha avuto picchi significativi. 
Voglio analizzare:
- Quando gli utenti iniziano a partecipare alle discussioni.
- Se c'è una relazione tra aumento dell'interesse (es. legato a prezzi in salita) e un aumento del coinvolgimento, attività o sentiment negativo nei commenti.

**Metodologia:**
Utilizzo la libreria pytrends per estrarre i dati da Google Trends, relativi alle query, cercando: **"Rolex Submariner/GMT/Datejust price"**

**Analisi dei dati raccolti:**
Andamento dell'interesse nel tempo su scala 0–100.
Identificazione dei picchi di interesse come proxy di eventi significativi (es. aumenti di prezzo, lanci di nuovi modelli, hype mediatico).
Esporto i dati in CSV.


In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt

pytrends = TrendReq(hl='en-US', tz=360)

# Rolex Submariner

Inizio con con il raccogliete dati per quanto riguarda il modello Rolex Submariner.

In [ ]:
model = 'Rolex Submariner price'
pytrends.build_payload([model], cat=0, timeframe='2013-01-01 2025-04-15', geo='', gprop='')

data_sub = pytrends.interest_over_time().reset_index()
data_sub['model'] = 'Submariner'

data_sub.to_csv("trend_rolex_submariner.csv", index=False)
print("File per Submariner salvato: trend_rolex_submariner.csv")
data_sub.head()


I dati ottenuti sono stati salvati nel file CSV **"trend_rolex_submariner.csv"**, che potrà essere successivamente utilizzato per:
- Visualizzazioni temporali
- Analisi di correlazione con le menzioni su Reddit
- Studio della popolarità nel tempo

In [ ]:
import time
print(" Attendo 10 minuti prima della prossima richiesta per evitare il blocco delle troppe richieste da Google (errore 429)")
time.sleep(600) 

# Rolex GMT

Inizio con con il raccogliete dati per quanto riguarda il modello Rolex GMT.

In [ ]:
model = 'Rolex GMT price'
pytrends.build_payload([model], cat=0, timeframe='2013-01-01 2025-04-15', geo='', gprop='')

data_gmt = pytrends.interest_over_time().reset_index()
data_gmt['model'] = 'GMT'

data_gmt.to_csv("trend_rolex_gmt.csv", index=False)
print("File per GMT salvato: trend_rolex_gmt.csv")
data_gmt.head()

In [ ]:
print(" Attendo 10 minuti prima della prossima richiesta per evitare il blocco delle troppe richieste da Google (errore 429)")
time.sleep(600) 

# Rolex Datejust

Inizio con con il raccogliete dati per quanto riguarda il modello Rolex Datejust.

In [ ]:
model = 'Rolex Datejust price'
pytrends.build_payload([model], cat=0, timeframe='2013-01-01 2025-04-15', geo='', gprop='')

data_dj = pytrends.interest_over_time().reset_index()
data_dj['model'] = 'Datejust'

data_dj.to_csv("trend_rolex_datejust.csv", index=False)
print("File per Datejust salvato: trend_rolex_datejust.csv")
data_dj.head()

# Confronto tra interesse globale e attività Reddit – Rolex Submariner

Adesso voglio confrontare l’evoluzione dell’interesse globale per il **Rolex Submariner**, misurato tramite Google Trends, con l’attività della community su Reddit.

**Grafico**
- **Linea blu** (asse sinistro): media annuale del trend Google con valori da 0 a 100.
- **Linea arancione tratteggiata** (asse destro): numero di commenti Reddit sul Submariner per anno

**Obiettivo:**
Verificare se l’aumento dell’interesse pubblico globale coincide con una maggiore partecipazione e discussione nella community del Social Media.

In [ ]:
# Carico i commenti Reddit
df_reddit = pd.read_csv("reddit_comments_rolex.csv")
df_reddit['created_utc'] = pd.to_datetime(df_reddit['created_utc'])
df_reddit['year'] = df_reddit['created_utc'].dt.year

# Filtr per modello "Submariner" e conto i commenti per anno
df_sub = df_reddit[df_reddit['model'].str.lower() == 'submariner'].copy()
reddit_yearly = df_sub.groupby('year').size().reset_index(name='num_comments')

# Carico il trend Google e calcolo la media annuale
df_trend = pd.read_csv("trend_rolex_submariner.csv")
df_trend['date'] = pd.to_datetime(df_trend['date'])
df_trend['year'] = df_trend['date'].dt.year
annual_df = df_trend.groupby('year')['Rolex Submariner price'].mean().reset_index(name='avg_trend')
annual_df.set_index('year', inplace=True)

merged = annual_df.merge(reddit_yearly, left_index=True, right_on='year', how='left')
merged.set_index('year', inplace=True)

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.set_title("Google Trends vs Commenti Reddit Rolex Submariner")

ax1.plot(merged.index, merged['avg_trend'], color='blue', marker='o', label='Trend medio (Google)')
ax1.set_xlabel("Anno")
ax1.set_ylabel("Google Trends (0–100)", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(merged.index, merged['num_comments'], color='orange', linestyle='--', marker='s', label='Commenti Reddit')
ax2.set_ylabel("N. commenti Reddit", color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()


Il grafico mostra una chiara crescita parallela a partire dal 2021, con un’esplosione dell’attività Reddit nel 2024–2025, coincidente con un forte incremento anche del trend su Google, quindi capisco che effettivamente l'interesse generale coincide con l'attività nella community di Reddit.

Faccio lo stesso confronto per gli altri due modelli di Rolex: **GMT** e **DateJust**

In [ ]:
model_name = "gmt"
trend_file = "trend_rolex_gmt.csv"
trend_column = "Rolex GMT price"
title = "Google Trends vs Commenti Reddit - Rolex GMT"

df_model = df_reddit[df_reddit['model'].str.lower() == model_name].copy()
reddit_yearly = df_model.groupby('year').size().reset_index(name='num_comments')

df_trend = pd.read_csv(trend_file)
df_trend['date'] = pd.to_datetime(df_trend['date'])
df_trend['year'] = df_trend['date'].dt.year
annual_df = df_trend.groupby('year')[trend_column].mean().reset_index(name='avg_trend')
annual_df.set_index('year', inplace=True)

merged = annual_df.merge(reddit_yearly, left_index=True, right_on='year', how='left')
merged.set_index('year', inplace=True)

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.set_title(title)
ax1.plot(merged.index, merged['avg_trend'], color='blue', marker='o', label='Trend medio (Google)')
ax1.set_xlabel("Anno")
ax1.set_ylabel("Google Trends (0–100)", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(merged.index, merged['num_comments'], color='green', linestyle='--', marker='s', label='Commenti Reddit')
ax2.set_ylabel("N. commenti Reddit", color='green')
ax2.tick_params(axis='y', labelcolor='green')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()


Anche qui grazie a questo confronto noto una correlazione positiva tra l’interesse generale e il coinvolgimento diretto della community Reddit, specialmente negli anni più recenti.

In [ ]:
model_name = "datejust"
trend_file = "trend_rolex_datejust.csv"
trend_column = "Rolex Datejust price"
title = "Google Trends vs Commenti Reddit - Rolex Datejust"

df_model = df_reddit[df_reddit['model'].str.lower() == model_name].copy()
reddit_yearly = df_model.groupby('year').size().reset_index(name='num_comments')

df_trend = pd.read_csv(trend_file)
df_trend['date'] = pd.to_datetime(df_trend['date'])
df_trend['year'] = df_trend['date'].dt.year
annual_df = df_trend.groupby('year')[trend_column].mean().reset_index(name='avg_trend')
annual_df.set_index('year', inplace=True)


merged = annual_df.merge(reddit_yearly, left_index=True, right_on='year', how='left')
merged.set_index('year', inplace=True)


fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.set_title(title)
ax1.plot(merged.index, merged['avg_trend'], color='blue', marker='o', label='Trend medio (Google)')
ax1.set_xlabel("Anno")
ax1.set_ylabel("Google Trends (0–100)", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(merged.index, merged['num_comments'], color='purple', linestyle='--', marker='s', label='Commenti Reddit')
ax2.set_ylabel("N. commenti Reddit", color='purple')
ax2.tick_params(axis='y', labelcolor='purple')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.show()

Ed anche per il Rolex Datejust il grafico mostra un aumento parallelo, soprattutto a partire dal 2022, notanto che l'interesse per questo modello è cresciuto non solo a livello di ricerche, ma anche in termini di discussione attiva nella community.

# Conclusione finale tra Google Trend e Commenti Reddit

In tutti e tre i casi, si osserva una relazione crescente nel tempo tra interesse di ricerca e volume di commenti.
L’impennata nei commenti Reddit tra il 2022 e il 2025 sembra rispecchiare, o in alcuni casi anticipare, i picchi osservati su Google.
Il modello GMT ha mostrato la crescita più esplosiva su Reddit, mentre il Datejust ha registrato un picco recente molto significativo.

L’obiettivo di questa analisi era verificare se esistesse una corrispondenza tra un potenziale aumento dei prezzi, dedotto dall’interesse crescente su Google, e l’incremento dell’interazione nei commenti su Reddit legato a tre modelli del brand di lusso dell'orologeria Rolex.

Tuttavia, la mancanza di uno storico accurato sui prezzi rappresenta un limite importante. Questo rende effettivamente impossibile stabilire con certezza una correlazione diretta tra le due variabili. Sebbene i dati mostrino una crescita parallela nello stesso arco temporale, **non mi è possibile concludere in modo definitivo che l’una influenzi l’altr**a.

# Analisi rete

Dopo aver esaminato l’andamento temporale dell’interesse per i modelli Rolex su Google e Reddit, il mio progetto prosegue con una nuova prospettiva: la **struttura della community**.

Voglio analizzare non solo quanto si parla dei modelli Rolex, ma anche come gli utenti si connettono tra loro in base a:

- Argomenti condivisi (es. modelli comuni)
- Discussioni sotto gli stessi thread
- Frequenza di partecipazione

# Costruzione del grafo bipartito: Utenti Reddit - Modelli Rolex

Adesso passo alla costruzione di un grafo bipartito per visualizzare le relazioni tra utenti Reddit e modelli Rolex.

Un insieme rappresenterà gli utenti Reddit, mentre l’altro insieme rappresenterà i modelli Rolex (Submariner, GMT, Datejust).
Un arco collega un utente a un modello se l’ha menzionato almeno una volta nei commenti

Tramite questo grafo posso vedere:
- Quali modelli sono più discussi (nodi ad alto grado)
- Quali utenti parlano di più modelli
- La presenza di community  attorno a specifici orologi

In [ ]:
import networkx as nx

with open("reddit_comments_data.json", "r") as comments_file:
    data = json.load(comments_file)

G = nx.Graph()

# Aggiungo nodi utente e modello al grafo
for comment in data:
    user = comment['author']
    model = comment['model']
    
    G.add_node(user, bipartite=0)   
    G.add_node(model, bipartite=1)  
    G.add_edge(user, model)         

user_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
model_nodes = set(G) - user_nodes


In [ ]:
nx.write_gexf(G, "grafo_bipartito_reddit_rolex.gexf")
print("Grafo esportato in: grafo_bipartito_reddit_rolex.gexf")

![](grafo_bipartito.png)

Il mio grafo bipartito è composto da:
- **Utenti Reddit** come un primo insieme di nodi
- **Modelli Rolex** (`Submariner`, `GMT`, `Datejust`) come secondo insieme di nodi

Un **arco collega** un utente a un modello se l’utente ha commentato almeno un post contenente riferimenti a quel modello.

- Il **modello GMT** (nodo rosso in basso a destra) è quello con il **maggior numero di utenti collegati**, indicando un interesse molto alto da parte della community.
- Il modello **Datejust** (nodo rosso a sinistra) ha un cluster compatto di utenti, ma meno esteso rispetto al GMT.
- Il grafo evidenzia **comunità di utenti** che si concentrano su uno o pochi modelli.
- Alcuni utenti sono connessi a più modelli, mostrando un comportamento trasversale nella discussione.

# Proiezione del grafo bipartito sugli utenti

Per capire meglio i comportamenti della mia community, effettuo la **proiezione del grafo bipartito** sugli utenti Reddit.  

In questa rete:
- Ogni **nodo** è un utente
- Un **arco** collega due utenti se hanno **commentato lo stesso modello Rolex**

Questo mi consente di:
- Rilevare **gruppi di utenti con interessi simili**
- Identificare utenti con **maggiore centralità** (più collegati ad altri)
- Visualizzare le **relazioni indirette** tra utenti

In [ ]:
G = nx.read_gexf("grafo_bipartito_reddit_rolex.gexf")

# Estraggo i nodi utente (bipartite = 0)
utenti = [n for n, d in G.nodes(data=True) if d["bipartite"] == 0]

G_utenti = nx.bipartite.projected_graph(G, utenti)

nx.write_gexf(G_utenti, "proiezione_utenti_rolex.gexf")
print("Proiezione utenti salvata: proiezione_utenti_rolex.gexf")


In [ ]:
print(f"Proiezione utenti Reddit:")
print(f"- Nodi (utenti): {G_utenti.number_of_nodes()}")
print(f"- Archi (connessioni): {G_utenti.number_of_edges()}")

![](grafo_proiezione_utenti.png)

# Analisi dei nodi più connessi nella rete 

Dopo aver costruito e proiettato il grafo bipartito sugli utenti Reddit, ho applicato un filtro basato sul **grado** dei nodi per identificare gli utenti più **"centrali"** nella rete.
Quindi ho voluto isolare e visualizzare gli **utenti con grado massimo**, ovvero quelli che hanno il maggior numero di connessioni e interazioni condivise con altri utenti attraverso i modelli Rolex.

- In **Gephi**, ho utilizzato il filtro **Topology > Degree Range** per selezionare i nodi con grado pari al valore massimo (in questo caso: `3215`).
- I nodi ottenuti rappresentano utenti **altamente connessi**, potenzialmente **hub** della conversazione.

In [ ]:
top_grado = 3215
utenti_con_grado_massimo = [(n, d) for n, d in G_utenti.degree if d == top_grado]

print(f"Utenti con grado {top_grado}: {len(utenti_con_grado_massimo)}")
for utente, grado in utenti_con_grado_massimo:
    print(f"- {utente} --> {grado} connessioni")


![](top_utenti_connessioni_3215.png)

# Analisi della mia rete utenti Rolex su Reddit

Dopo aver costruito la proiezione degli utenti a partire dal grafo bipartito, procedo con un'analisi quantitativa della rete ottenuta.

# Grado Minimo, Massimo e Grado Medio

Un grado massimo elevato può indicare la presenza di utenti estremamente attivi, mentre un grado minimo può segnalare utenti poco connessi o marginali.
Mentre il grado medio rappresenta il livello medio di connessione degli utenti nella rete.

In [ ]:
degrees = list(G_utenti.degree())

grado_minimo = min(degrees, key=lambda x: x[1])[1]
print("Grado minimo:", grado_minimo)

grado_massimo = max(degrees, key=lambda x: x[1])[1]
print("Grado massimo:", grado_massimo)

grado_medio = sum(d for n, d in degrees) / len(degrees)
print("Grado medio:", grado_medio)


# Densità

La densità misura quanto il grafo è connesso rispetto al massimo possibile, ed è un valore che va da 0 ad 1. 
- 0 significa nessuna connessione tra i nodi (grafico completamente scollegato, solo nodi isolati).
- 1 significa che tutti i nodi sono connessi tra loro (grafo completamente pieno, ogni utente collegato con tutti gli altri).
Quindi per valori alti, vicino all'1, il grafo sarà molto denso e con forte interazione; mentre per valori bassi la rete sarà più sparsa.

In [ ]:
densita = nx.density(G_utenti)
print("Densità del grafo:", densita)


La densità appena trovata, ovvero 0.5729461170044026, è relativamente alta, considerando che il grafo ha un numero considerevole di nodi. Posso dire che è **compatta e ben connessa**, in cui oltre la metà delle possibili connessioni sono effettivamente presenti, quindi mi indica che molti utenti si relazionano tra loro.
Quindi effettivamente c'è un forte interesse condiviso tra gli utenti nei confronti dei modelli Rolex analizzati.

# Moda del grado
La moda è il grado che si ripete più frequentemente: indica il livello di attività "tipico" nella rete.

In [ ]:
degree_counts = {}
for _, degree in degrees:
    if degree in degree_counts:
        degree_counts[degree] += 1
    else:
        degree_counts[degree] = 1

most_common_degree = max(degree_counts, key=degree_counts.get)
print(f"Grado più comune (moda): {most_common_degree}")


# Mediana del grado

La mediana aiuta a capire la distribuzione dei gradi. Se è molto più bassa della media, indica pochi nodi hub molto potenti.

In [ ]:
gradi = [d for _, d in degrees] 
gradi_ordinati = sorted(gradi) 
lunghezza = len(gradi_ordinati) 

if lunghezza % 2 == 1:
    mediana = gradi_ordinati[lunghezza // 2]
else:
    indice_medio = lunghezza // 2
    mediana = (gradi_ordinati[indice_medio - 1] + gradi_ordinati[indice_medio]) / 2

print(f"Mediana dei gradi: {mediana}")


Questo data significa che il 50% degli utenti ha un grado (numero di connessioni) inferiore o uguale a 2191, mentre l’altro 50% ha un grado superiore o uguale.

Poiché la **mediana è abbastanza alta**, considerando il massimo di 3215 connessioni possibili, deduco che:
- La rete è ben connessa, quinndi la maggior parte degli utenti ha un elevato numero di connessioni verso altri utenti.
- Non è una rete dominata da pochissimi super-nodi (hub), al contrario, molti utenti partecipano attivamente e stabiliscono numerosi collegamenti.

Se la mediana fosse stata molto più bassa rispetto alla media o al grado massimo, avremmo avuto una rete più disomogenea, con pochi utenti fortemente centrali e molti periferici.

# ECDF (Empirical Cumulative Distribution Function) dei gradi

La funzione di distribuzione cumulativa empirica (ECDF) permette di visualizzare come sono distribuiti i gradi dei nodi nella rete degli utenti che commentano modelli Rolex su Reddit.
L'asse delle ordinate rappresenta la frazione cumulativa di utenti con grado minore o uguale a un certo valore.

In [ ]:
import numpy as np

frazione_cumulativa = np.arange(1, len(gradi_ordinati) + 1) / len(gradi_ordinati)

plt.figure(figsize=(8,6))
plt.plot(gradi_ordinati, frazione_cumulativa, marker='o', markersize=3, linestyle='-', color='purple')
plt.xlabel('Grado del nodo')
plt.ylabel('Frazione cumulativa')
plt.title('ECDF dei gradi degli utenti (Reddit Rolex)')
plt.grid(True)
plt.show()


Dal grafico ECDF osservo che il network degli utenti Reddit interessati a Rolex presenta una struttura piuttosto coesa, con la maggior parte degli utenti è connessa ai tre modelli tramite numerose interazioni. Solo una piccola parte risulta poco connessa o isolata. Difatti, più del 70% degli utenti ha un grado superiore a 1000, indicando una forte partecipazione e interazione tra gli utenti.
Inoltre, la presenza di nodi con grado molto elevato suggerisce l’esistenza di hub centrali, ovvero utenti particolarmente attivi o influenti all’interno della rete.

# Componenti connesse

Voglio sapere se tutti gli utenti sono **"parte della stessa rete"** o se ci sono "isole separate".

In [ ]:
connected_components = list(nx.connected_components(G_utenti))

for i, component in enumerate(connected_components, 1):
    num_utenti = len(component)
    print(f"Componente: {i} --> {num_utenti} utenti:")
    
    # Scrivo i primi 10 utenti
    utenti_lista = list(component)
    print(", ".join(utenti_lista[:10]))
    if num_utenti > 10:
        print(f"+ altri {num_utenti-10} utenti")

num_components = len(connected_components)
print(f"\nNumero totale di componenti connesse: {num_components}")


**Avendo una sola componente connessa** significa che l'intera rete è strutturalmente unita, quindi la dimensione della componente più grande: 3216 nodi, ovvero pari al numero totale di utenti.
Non ci sono nodi isolati o gruppetti separati, pertanto l'interazione può propagarsi liberamente tra qualsiasi coppia di utenti nella rete.

# MISURE DI CENTRALITÀ

# Degree centrality 
La degree centrality è una misura fondamentale per capire quanto un nodo è connesso nel network rispetto agli altri.
Più un nodo ha connessioni, maggiore sarà la sua centralità e quindi la sua influenza nella rete.

Nel mio progetto:
- Gli utenti Reddit più centrali sono quelli che hanno commentato su più modelli Rolex o che hanno collegamenti con più utenti simili.
- Un valore di degree centrality più alto indica utenti più attivi o influencer all'interno della community Rolex

In [ ]:
degree_centrality = nx.degree_centrality(G_utenti)

# Ordino i nodi per centralità decrescente
sorted_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)

print("Primi 100 utenti per Degree Centrality:")
for node in sorted_nodes[:100]:
    print(f"Nodo: {node}, Degree Centrality: {degree_centrality[node]:}") 


Grazie alla mia Degree Centrality, noto che tanti utenti (ad esempio saloulkm, sporturaws, Euphoric_Ad_6091, ecc.) hanno una degree centrality pari quasi ad 1 (arrotondado).

**Degree Centrality = 1** significa che questi utenti sono collegati a tutti gli altri utenti della rete, o quasi. Sono super connessi, dei veri e propri hub centrali!
Poi si nota una caduta nei valori successivi, ad esempio:
- Utenti come Morake122, legendaryrhirrany, hoo_haaa, ecc.(comunque sempre nei **primi 100**) hanno una degree centrality di 0.8547,
Quindi sono ancora molto centrali, ma non completamente collegati a tutti gli altri nodi.

In [ ]:
print("Ultimi 50 utenti per Degree Centrality:")
for node in sorted_nodes[-50:]:
    print(f"Nodo: {node}, Degree Centrality: {degree_centrality[node]:.5f}") 

Analizzando invece i **50 utenti con Degree Centrality più bassa**, osservo che il loro valore si aggira intorno a 0.24479, indicando una posizione marginale nella rete. 

Questi utenti risultano meno connessi rispetto agli hub principali, suggerendo un coinvolgimento limitato o focalizzato su pochi modelli Rolex. 
Visivamente, questi nodi si posizionano ai margini della rete, lontani dal nucleo più denso di connessioni.

In [ ]:
top_10_nodes = sorted_nodes[:10]
bottom_10_nodes = sorted_nodes[-10:]

nodi_top = [node for node  in top_10_nodes]
valori_top = [degree_centrality[node] for node in top_10_nodes]

nodi_bottom = [node for node in bottom_10_nodes]
valori_bottom = [degree_centrality[node] for node in bottom_10_nodes]

fig, ax = plt.subplots(figsize=(14, 6))

ax.barh(nodi_top, valori_top, color='royalblue', label="Top 10 utenti")

ax.barh(nodi_bottom, valori_bottom, color='lightcoral', label="Ultimi 10 utenti")

ax.invert_yaxis()
plt.xlabel('Degree Centrality')
plt.title('Top 10 vs Ultimi 10 utenti - Degree Centrality Reddit Rolex')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
G_copy = G_utenti.copy()

for node in G_copy.nodes():
    if node in top_10_nodes:
        G_copy.nodes[node]['tipo'] = 'top'
    elif node in bottom_10_nodes:
        G_copy.nodes[node]['tipo'] = 'bottom'
    else:
        G_copy.nodes[node]['tipo'] = 'other'

nx.write_gexf(G_copy, "grafo_reddit_rolextop_bottom.gexf")

print("Grafo esportato come grafo_reddit_rolextop_bottom.gexf")


![](top_bottom_utenti.png)

# Distribuzione dei nodi

Dopo a aver isolati i nodi dal resto vedo che:
- I nodi **verdi** rappresentano i **Top 10 utenti**, quelli più centrali, ad alto degree centrality. Sono molto connessi tra di loro e anche con altri nodi. Alcuni sono veri e propri "hub" che tengono unita la rete.
- I nodi **rossi** rappresentano i **Bottom 10 utenti**, quelli meno centrali, con pochi collegamenti. Sono meno connessi, spesso connessi solo a uno o pochi nodi Top o tra di loro debolmente.

Per quanto riguarda le connessioni, si nota che molti archi partono dai nodi "Top" verso altri nodi, mentre quelli "Bottom" hanno meno legami forti.

# Analisi della Betweenness Centrality per il mio grafo Reddit Rolex

La betweenness centrality misura quanto spesso un nodo si trova sul cammino più breve tra altri due nodi nella rete.

Se un nodo ha alta betweenness, significa che è un ponte importante tra gruppi di utenti.
Se ha bassa betweenness (vicino a 0), non è essenziale per collegare altri nodi: è "laterale", non passa attraverso di lui l'informazione.

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G_utenti)

Rispetto alla degree centrality calcolare la betweenness centrality richiede molto più tempo. 

**Degree centrality** va a contare solo quanti archi ha un nodo, quindi l'operazione è abbastanza veloce, perchè basta scorrere tutti i vicini.
**Betweenness centrality**, invece va a calcolare tutti i cammini minimi tra ogni coppia di nodi. Per ogni nodo, vede quanto spesso passa attraverso i cammini minimi. 

Questo significa risolvere migliaia di shortest paths e quindi il costo computazionale sarà molto più alto.

In [ ]:
# Ordino i nodi per betweenness decrescente
sorted_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)

print("Lista completa degli utenti per Betweenness Centrality:")
for node, betweenness in sorted_betweenness:
    print(f"Nodo: {node}, Betweenness Centrality: {betweenness:}")

In [ ]:
print("Top 10 utenti per Betweenness Centrality:")
for node, betweenness in sorted_betweenness[:10]:
    print(f"Nodo: {node}, Betweenness Centrality: {betweenness:}")

print("\nUltimi 10 utenti per Betweenness Centrality:")
for node, betweenness in sorted_betweenness[-10:]:
    print(f"Nodo: {node}, Betweenness Centrality: {betweenness:}")

# Closeness Centrality

La closeness centrality misura quanto un nodo è "vicino" a tutti gli altri nodi della rete.
Un valore alto indica che l'utente può raggiungere rapidamente gli altri utenti attraverso un numero minimo di passaggi, quindi è ben connesso "globalmente".

Voglio calcolare la closeness centrality per tutti gli utenti della mia rete, ordinandoli dal più "vicino" al più "lontano".

In [ ]:
closeness_centrality = nx.closeness_centrality(G_utenti)

# Ordino i nodi per closeness decrescente
sorted_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)

print("Tutti gli utenti ordinati per Closeness Centrality:")
for node, closeness in sorted_closeness:
    print(f"Nodo: {node}, Closeness Centrality: {closeness:}")


In [ ]:
print("Top 10 utenti per Closeness Centrality:")
for node, closeness in sorted_closeness[:10]:
    print(f"Nodo: {node}, Closeness Centrality: {closeness:}")

print("\nUltimi 10 utenti per Closeness Centrality:")
for node, closeness in sorted_closeness[-10:]:
    print(f"Nodo: {node}, Closeness Centrality: {closeness:}")

L'analisi della closeness centrality mostra che nella mia rete esiste un piccolo gruppo di utenti che può raggiungere rapidamente il resto della rete, facilitando così la comunicazione e la propagazione delle informazioni. 
Però, ho trovato anche la presenza di utenti con valori bassi suggerisce anche l'esistenza di aree marginali della rete.

# Transitività

La transitività è una misura che indica quanto la rete sia coesa, ossia quanto è probabile che due nodi connessi a un nodo comune siano anche connessi tra loro.
È calcolata tramite il coefficient di clustering globale, che rappresenta la proporzione di triangoli effettivamente presenti rispetto a quelli potenzialmente formabili nella rete.

- Alta transitività: la rete ha una struttura fortemente coesa, formata da gruppi o comunità ben definite.
- Bassa transitività: la rete è più frammentata, con meno connessioni triangolari e una struttura quindi meno compatta.

In [ ]:
transitivity = nx.transitivity(G_utenti)
print(f"Transitività del grafo utenti: {transitivity:.4f}") 

L'analisi della transitività mi ha restituito un valore di 0.9119, indicando un **livello molto alto di coesione tra gli utenti.** 
Questo significa che se due utenti sono collegati a un terzo utente, è estremamente probabile che siano anche collegati tra loro. 

La rete mostra una forte tendenza alla formazione di gruppi chiusi e comunità compatte, suggerendo un'interazione intensiva e specifica su temi comuni relativi al mondo Rolex su Reddit.

# Clustering medio

Il clustering medio misura quanto, in media, i vicini di un nodo sono tra loro connessi.
Serve a capire se nella rete gli utenti tendono a formare piccoli gruppi o community compatte.

In [ ]:
average_clustering = nx.average_clustering(G_utenti)
print(f"Clustering medio della rete utenti: {average_clustering:}")


Il **clustering medio** della rete è risultato essere pari a 0.9518, un valore anch'esso molto elevato che evidenzia la **forte tendenza degli utenti a formare gruppi chiusi**. 
All'interno della mia rete, i vicini di un utente sono spesso connessi anche tra loro, dando origine a comunità molto compatte. Tale struttura favorisce una rapida circolazione delle informazioni all'interno dei gruppi e suggerisce un elevato livello di omogeneità tematica e di interazione sociale.

![](Clustering_Coefficient.png)
![](Clustering_Coefficient_2.png)
![](report_clustering_coefficient.png)

In **Gephi** ho colorato i nodi in base al valore del loro clustering coefficient: nodi con valori simili vengono raggruppati e colorati in modo simile.

La maggioranza dei nodi ha un c**lustering coefficient molto alto**, 1.0 ed è indicato dal colore fucsia, ed è pari all' **84,61**% dei nodi.
Questo significa che la maggior parte degli utenti forma "gruppi chiusi", dove ogni utente è molto collegato agli altri utenti del proprio gruppo.
Gli altri piccoli gruppi colorati (blu, verde, arancione) hanno valori leggermente più bassi di clustering, distribuiti vicino a 0.5-0.8, sono minoranze quindi ed ogni gruppo rappresenta circa il **2%-6%** degli utenti.

Dal grafico della distribuzione invece posso vedere che quasi tutto il conteggio si concentra vicino a 1, confermando che la rete è altamente clusterizzata, quindi la stragrande maggioranza dei nodi forma triangoli chiusi, quindi gli utenti tendono a connettersi con amici di amici.

L'analisi del clustering coefficient mostra che nella mia rete, ho una struttura fortemente comunitaria della rete, con pochi utenti che agiscono come ponti tra comunità diverse, che posso individuarli come le "code" esterne nel mio grafo.

# Cammino Minimo Medio

Il cammino minimo medio rappresenta la distanza media più breve tra tutte le coppie di nodi nel grafo.
Un valore vicino a 1 indica che quasi tutti gli utenti sono direttamente connessi tra loro oppure che bastano pochissimi passaggi per raggiungere chiunque altro.

In [ ]:
average_shortest_path_length = nx.average_shortest_path_length(G_utenti)

print("Cammino minimo medio:", average_shortest_path_length)

L'analisi del **cammino minimo medio** mostra che in media servono circa **1,43 passaggi** per collegare un qualsiasi utente a un altro nella mia rete. 
Questo valore molto basso evidenzia una struttura fortemente coesa, in cui l'informazione o le interazioni possono propagarsi rapidamente tra gli utenti.

Questo dimostra ulteriormente con quello visto finora (clustering e transitività alta, grafi compatti), quindi **la mia rete di utenti è davvero connessa**!

# Similiarità

La similarità misura quanto due utenti sono "vicini" tra loro in base ai modelli di orologi di cui parlano nei commenti.
In questo caso, ho utilizzato il coefficiente di similarità di Jaccard per confrontare i modelli associati agli utenti:

- Valore 0: gli utenti non hanno mai commentato lo stesso modello di orologio.
- Valore tra 0 e 1: maggiore è il valore, più alta è la similarità, cioè condividono modelli in comune.
- Valore 1: gli utenti hanno commentato esattamente gli stessi modelli.

Per cacolarla, carico il mio dizionario dal file json reddit_comments_data.json, dove sono stati estratti gli utenti e i modelli di orologi da loro commentati.
Per ogni coppia di utenti, calcolo il coefficiente di Jaccard, che confronta l'intersezione e l'unione dei modelli commentati.
Infine, i risultati li salvo in una matrice di similarità.

In [ ]:
from itertools import combinations

def jaccard_similarity(list1, list2):
    intersection = len(set(list1).intersection(set(list2)))
    union = len(set(list1).union(set(list2)))
    return intersection / union if union != 0 else 0.0

with open('reddit_comments_data.json', 'r') as f:
    reddit_comments = json.load(f)

# Costruisco dizionario utente -> lista di modelli commentati
user_model_dict = {}
for comment in reddit_comments:
    user = comment['author']
    model = comment['model']
    if user in user_model_dict:
        user_model_dict[user].append(model)
    else:
        user_model_dict[user] = [model]

users = list(user_model_dict.keys())
num_users = len(users)

similarity_matrix = np.zeros((num_users, num_users))

# Calcolo similarità per tutte le coppie
for i, user1 in enumerate(users):
    for j, user2 in enumerate(users):
        if j > i:
            models_user1 = set(user_model_dict[user1])
            models_user2 = set(user_model_dict[user2])
            similarity = jaccard_similarity(models_user1, models_user2)
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  

np.savetxt('user_similarity_matrix.txt', similarity_matrix, delimiter='\t')

# Stampo alcune similarità
for i, user1 in enumerate(users):
    for j, user2 in enumerate(users):
        if j > i and similarity_matrix[i, j] > 0:
            print(f"Similarità tra {user1} e {user2}: {similarity_matrix[i, j]:}")


Visualizzo la **matrice di similarità** come una mappa a colori.

In [ ]:
import seaborn as sns

similarity_matrix = np.loadtxt('user_similarity_matrix.txt', delimiter='\t')

# Seleziono solo i primi 100 utenti per una visibilità migliore
subset_size = 100
small_matrix = similarity_matrix[:subset_size, :subset_size]

plt.figure(figsize=(12, 10))
sns.heatmap(small_matrix, cmap='viridis', cbar=True)
plt.title('Mappa di Similarità tra i primi 100 utenti (Jaccard)', fontsize=16)
plt.xlabel('Utenti')
plt.ylabel('Utenti')
plt.tight_layout()
plt.show()

Ogni asse rappresenta un utente.

Il colore di ogni cella, indica quanto sono simili tra loro l'utente "i" e l'utente "j", usando il coefficiente di similarità di Jaccard.

- Colori più chiari, come il giallo, indicano un' altissima similarità (vicina a 1.0), cioè che gli utenti hanno commentato molti moelli in comune.
- Colori scuri, blu o viola, invece indicano un bassa similarità (vicina a 0.0), quindi gli utenti hanno commentato modelli diversi tra loro.

Adesso creo un nuovo **grafo degli utenti basato però sulla similarità**.

Collego solo gli utenti che sono abbastanza simili, ad esempio se Jaccard > 0.5

Prima però carico fil file json dove per ogni commento c'è un "author", e questi author sono proprio gli utenti che vado ad estrarre e costruisco la mia lista users.

In [ ]:
with open('reddit_comments_data.json', 'r') as f:
    reddit_comments = json.load(f)

user_model_dict = {}
for comment in reddit_comments:
    user = comment['author']
    model = comment['model']
    if user in user_model_dict:
        user_model_dict[user].add(model)
    else:
        user_model_dict[user] = {model}

filtered_users = [user for user, models in user_model_dict.items() if len(models) >= 2]

print(f"Numero di utenti che hanno commentato almeno 2 modelli: {len(filtered_users)}")

In [ ]:
similarity_matrix = np.loadtxt('user_similarity_matrix.txt', delimiter='\t')

all_users = list(user_model_dict.keys())

indices = [all_users.index(user) for user in filtered_users]
filtered_similarity_matrix = similarity_matrix[np.ix_(indices, indices)]

In [ ]:
G_similarity_filtered = nx.Graph()

for user in filtered_users:
    G_similarity_filtered.add_node(user)

# Aggiungo gli archi solo se la similarità supera una soglia alta pari a 0.7
threshold = 0.7

for i, user1 in enumerate(filtered_users):
    for j, user2 in enumerate(filtered_users):
        if j > i and filtered_similarity_matrix[i, j] > threshold:
            G_similarity_filtered.add_edge(user1, user2, weight=filtered_similarity_matrix[i, j])

nx.write_gexf(G_similarity_filtered, "grafo_similarita_utenti_filtrato.gexf")

print("Grafo filtrato creato e salvato come 'grafo_similarita_utenti_filtrato.gexf'")

Ho costruito una nuova rete dove i nodi rappresentano gli utenti Reddit, e gli archi esistono solo se la similarità tra gli utenti, calcolata tramite il coefficiente di Jaccard sui modelli Rolex commentati, supera una soglia di 0.7. 
Questa rete consente di visualizzare e analizzare cluster di utenti con gusti molto simili all'interno della community.

![](similiarità_utenti.png)

- **Nodi totali:** 495 utenti Reddit (in alto a destra).
- **Archi totali:** 37.903 connessioni di similarità.
- **4 componenti principali:**
  - Blu: 40,4%
  - Rosso: 32,7%
  - Verde: 13,4%
  - Rosa: 13,5%

La mia rete evidenzia **quattro community principali** di utenti Reddit, ciascuna con **comportamenti simili** in termini di commenti sui modelli.
- Gli utenti **dello stesso cluster** (stesso colore) tendono ad avere gusti simili: hanno commentato **gli stessi due modelli Rolex**.
- La rete è **densa**, segno che questi utenti sono altamente interconnessi tra loro.
- La presenza di **più componenti** mostra che ci sono sottogruppi abbastanza omogenei da essere distinti dagli altri, anche con soglie di similarità elevate.

Questa visualizzazione conferma che gli utenti Reddit che commentano più modelli Rolex tendono a formare **gruppi chiusi e coesi**.

# Community Detection

Per identificare sottogruppi di utenti Reddit con comportamenti simili, adesso applico l'**algoritmo di rilevamento comunità basato sulla modularità Greedy**.  
Con questo metodo cerco di massimizzare la **modularità**, un indice che misura quanto bene una rete può essere suddivisa in comunità coese.

In [ ]:
import networkx as nx
from networkx.algorithms import community

G = nx.read_gexf("grafo_similarita_utenti_filtrato.gexf")

# Rilevamento delle community con greedy modularity
communities = community.greedy_modularity_communities(G)

for i, group in enumerate(communities):
    print(f"Community {i+1} ({len(group)} utenti): {list(group)[:20]}...")  # Mostra solo i primi 20 utenti per ogni gruppo

L’algoritmo ha identificato **4 community**, ognuna composta da utenti che presentano **gusti simili**, ovvero hanno commentato **gli stessi modelli Rolex**.

# MACHINE LEARNING

# User Clustering by Comments

## Creazione documento testuale per utuente basato sui commenti

Creo il documento testuale per utente:
- Andrò a caricare i dati Reddit contenenti i commenti su modelli Rolex.
- Creo un dizionario utente --> lista di commenti
- Ogni utente sarà rappresentato da un documento.

In [ ]:
with open("reddit_comments_data.json", "r") as f:
    rolex_data = json.load(f)

# Dizionario utente -> lista di commenti
user_comments = defaultdict(list)
for entry in rolex_data:
    user = entry["author"].lower()
    comment = entry["text"]
    user_comments[user].append(comment)

# Genero documento testuale unico per ciascun utente
user_ids = list(user_comments.keys())
document4user = [" ".join(comments) for comments in user_comments.values()]

# Stampo un esempio di utente e il suo documento
user_ids[7], document4user[7]

## Embedding dei commenti usando SpaCy

Uso il modello en_core_web_md di spaCy per trasformare ogni documento in un vettore denso (embedding), dove ogni riga della matrice corrisponde a un utente e rappresenta il “profilo semantico” dei modelli menzionati.

In [ ]:
import spacy
import numpy as np

# Carico modello SpaCy in inglese 
nlp = spacy.load("en_core_web_md")

# Calcolo i vettori dei documenti
X = np.array([nlp(doc).vector for doc in document4user])

## PCA (Riduzione della dimensionalità) 

Adesso applico la PCA per ridurre la dimensionalità dei vettori pur mantenendo almeno il 90% della varianza, per non perdere informazioni e così rendo rendo dati più maneggevoli per l’algoritmo di clustering.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, Normalizer

# Applico PCA mantenendo almeno il 90% della varianza
pca = PCA(n_components=0.9, svd_solver='full')
normalizer = Normalizer(copy=False)
lsa = make_pipeline(StandardScaler(), pca, normalizer)

X = lsa.fit_transform(X)

explained_variance = pca.explained_variance_ratio_.sum()
print("Explained variance utilizzando PCA: {}%".format(int(explained_variance * 100)))

In [ ]:
# Stampo la forma della matrice X
X.shape

Ho quinndi **3216 utenti/documenti** (che corrispondno alle righe della matrice), dove ciascuno di essi è rappresentato da un **vettore di 29 dimensioni** (le colonne), ovvero 29 componenti principali selezionate automaticamente dalla PCA per mantenere almeno il 90% della varianza.

## Clustering con k-means

Eseguo l’algoritmo K-Means per raggruppare utenti simili.

In [ ]:
from sklearn.cluster import KMeans

# Eseguo clustering
kmeans = KMeans(n_clusters=4, random_state=42) # Ho scelto il numero di cluster pari a 4 poi controllerò con la Distorsione
labels = kmeans.fit_predict(X)

In [ ]:
pca_vis = PCA(n_components=2)
X_vis = pca_vis.fit_transform(X)
                              
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_vis[:, 0], X_vis[:, 1], c=labels, cmap="tab10", alpha=0.7)
plt.title("User Clustering by Comments - Rolex")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(scatter, label="Cluster")
plt.grid(True)
plt.show()

km = KMeans(n_clusters = 4, init='k-means++', max_iter=100, n_init=1)
km.fit(X)

Il grafico rappresenta una riduzione dimensionale tramite **PCA** dei vettori testuali dei commenti degli utenti Rolex.

Ogni punto è un utente, posizionato nello spazio in base alla somiglianza nei contenuti dei commenti.
Il colore indica il cluster a cui l’utente è stato assegnato dall’**algoritmo K-Means**.
-	Utenti nello stesso cluster hanno commenti simili.
-	Cluster ben separati visivamente indicano gruppi omogenei, con comportamenti distinti.

## Distorsione 

Voglio analizzare l'andamento della distorsione al variare del numero di cluster da identificare.

Calcolo la distorsione per diversi valori di k (ovvero numero di cluster), così trovo il punto in cui l’attributo **inertia_** inizia a decrescere lentamente. 
Quel punto rappresenta un buon numero di cluster, bilanciando accuratezza e semplicità.

In [ ]:
distortions = []
K_range = range(2, 40, 2)  # Provo con un range ampio per vedere l'andamento

for k in K_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)  
    distortions.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K_range, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.grid(True)
plt.show()

Il grafico del metodo mostra che l’inertia decresce rapidamente fino a circa 4 cluster, per poi appiattirsi gradualmente. 
Quindi con **k = 4** è un buon compromesso tra semplicità e qualità del clustering.

## Silhoutte Coefficient 

Per verificare la **qualità** del clustering utilizzo la silhouette. La silhouette misura quanto un oggetto e' simile al cluster di appartenenza rispetto agli altri cluster.

Il Silhouette varia tra -1 e 1.
- Valori vicini a 1 indicano cluster ben separati.
- Valori vicini a 0 indicano sovrapposizione.
- Valori negativi indicano che molti punti sono assegnati al cluster sbagliato.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette = silhouette_score(X, labels)
print(f"Silhouette Coefficient: {silhouette}")

## Visualizzazione dei commenti per cluster

Tramite il mio dizionario user_comments, visualizzo i commenti raggruppati.

In [ ]:
for c in sorted(set(labels)):
    print(f"\n--- Cluster {c} ---")
    users_in_cluster = [u for u, lbl in zip(user_ids, labels) if lbl == c]
    for u in users_in_cluster[:5]:  # Mostro solo i primi 5 utenti per cluster
        print(f"{u}: {user_comments.get(u, '')[:150]}...")

# Analisi dei Modelli più Citati per Cluster

Adesso voglio **associare ogni utente ad un cluster** e analizzre quali parole o hashtag sono più frequenti all’interno di ogni gruppo.

- Considero ogni utente come un “documento” composto dai modelli Rolex menzionati nei commenti.
- Voglio anche vedere quali modelli vengono citati più frequentemente in ogni cluster.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=2)  
pipeline = make_pipeline(StandardScaler(), pca)
X = pipeline.fit_transform(X)  
X = np.array([nlp(doc).vector for doc in document4user])

In [ ]:
from collections import Counter

km = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
km.fit(X)

# Creo IL dizionario user_id --> cluster
user_to_cluster = dict(zip(user_ids, km.labels_))

# Aggiungo anche i modelli associati a ciascun utente
user_profile = {
    k: {
        'group': v,
        'models': document4user[i].split()
    }
    for i, (k, v) in enumerate(user_to_cluster.items())
}

modelli_rolex ={ 'gmt', 'submariner', 'datejust', 'daytona', 
                'yachtmaster', 'explorer', 'sea-dweller', 'sky-dweller'
                'day-date', 'milgauss', 'air-king', 'cellini',
                'oyster-perpetual', 'pepsi', 'batman', 'hulk', 'rootbeer'
                'airking', 'tudor', 'black-bay', 'pelagos', 'heritage'
                }

def hashtags_group(profiles, group, common=10):

    all_models = defaultdict(set)
    
    for user, info in profiles.items():
        if info['group'] == group:
            for model in set(info['models']):  
                model_clean = model.lower()
                if model_clean in modelli_rolex: 
                    all_models[model_clean].add(user)
    

    model_counts = {model: len(users) for model, users in all_models.items()}
    

    return Counter(model_counts).most_common(common)

for g in range(4):
    print(f"Cluster {g}:")
    print(hashtags_group(user_profile, g))
    print("-" * 40)

Ho quindi **analizzato quali modelli tra tutti i Rolex vengono menzionati più frequentemente** da ciascun gruppo di utenti, cluster, secondo l'output dell'algoritmo di clustering.
Questo mi aiuta a capire se ci sono gruppi di utenti con gusti e interessi simili verso determinati modelli Rolex.

### Cluster 0
- **Modelli più menzionati:** gmt, datejust, explorer, pepsi, tudor, batman, daytona, hulk
- **Interpretazione:** Gruppo variegato con interessi distribuiti su vari modelli popolari.
- **Possibile profilo:** Appassionati generici di Rolex, con buona conoscenza del brand e interesse sia per modelli classici (es. datejust) sia sportivi (gmt, daytona, batman).

### Cluster 1
- **Modelli più menzionati:** gmt, submariner, pepsi, explorer, daytona
- **Interpretazione:** Cluster ristretto, focalizzato su pochi modelli iconici.
- **Possibile profilo:** Utenti più “mainstream” o nuovi appassionati, interessati ai modelli Rolex più noti.

### Cluster 2
- **Modelli più menzionati:** gmt, submariner, tudor
- **Interpretazione:** Molto compatto, con una forte concentrazione su due modelli principali.
- **Possibile profilo:** Fan dei modelli professionali da immersione o sportivi, con gusti più focalizzati.

### Cluster 3
- **Modelli più menzionati:** gmt, submariner, tudor, explorer, daytona, batman, hulk, yachtmaster
- **Interpretazione:** Il cluster più ampio e diversificato.
- **Possibile profilo:** Utenti con elevata conoscenza del brand, probabilmente esperti o collezionisti. Menzionano anche modelli più “di nicchia” (yachtmaster).


# Sentiment Analysis dei commenti per modello di orologio

La **sentiment analysis** è una tecnica utile per valutare l’opinione espressa dagli utenti nei commenti. In questo caso, la applico ai commenti Reddit relativi a diversi modelli di orologi Rolex, per capire **quali modelli suscitano reazioni più positive o negative** nella community.

Utilizzerò la libreria **TextBlob** per calcolare il sentiment, positivo o negativo, dei commenti associati a ciascun modello.

In [ ]:
import json
from textblob import TextBlob
import matplotlib.pyplot as plt

with open('reddit_comments_data.json', 'r') as f:
    data = json.load(f)

def classify_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity < 0:
        return "Negativo"
    else:
        return "Positivo"

# Dizionario modello -> lista di commenti
model_comments = {}
for entry in data:
    model = entry.get('model')
    text = entry.get('text', '')
    if model and text:
        model_comments.setdefault(model, []).append(text)

# Analisi del sentiment per modello
model_sentiments = {}
for model, comments in model_comments.items():
    sentiment_counts = {"Positivo": 0, "Negativo": 0}
    for comment in comments:
        sentiment = classify_sentiment(comment)
        sentiment_counts[sentiment] += 1
    model_sentiments[model] = sentiment_counts

In [ ]:
for model, sentiments in model_sentiments.items():
    print(f"Modello: {model}")
    print(f"  Commenti positivi: {sentiments['Positivo']}")
    print(f"  Commenti negativi: {sentiments['Negativo']}")
    print("-" * 40)

models = list(model_sentiments.keys())
positive = [model_sentiments[m]["Positivo"] for m in models]
negative = [model_sentiments[m]["Negativo"] for m in models]

x = range(len(models))
plt.figure(figsize=(12, 6))
plt.bar(x, positive, color='green', label='Positivi')
plt.bar(x, negative, bottom=positive, color='red', label='Negativi')
plt.xticks(x, models, rotation=45)
plt.ylabel("Numero di commenti")
plt.title("Sentiment dei commenti per modello Rolex")
plt.legend()
plt.tight_layout()
plt.show()

Noto che il modello **GMT** riceve di gran lunga il maggior numero di commenti, indicando un coinvolgimento molto alto nella mia community.
Si vede che la **maggioranza dei commenti è positiva per tutti e tre i modelli.**

# Analisi del Sentiment per il modello "Submariner"

Adesso andrò a selezionare tutti i commenti associati al modello **Submariner** per fare un'esempio.

In [ ]:
model_to_analyze = "submariner"
model_comments = [entry for entry in data if entry["model"].lower() == model_to_analyze.lower()]

def classify_comment_sentiment(comment_text):
    analysis = TextBlob(comment_text)
    polarity = analysis.sentiment.polarity
    return "Positivo" if polarity >= 0 else "Negativo"

comment_sentiments = []
for entry in model_comments:
    sentiment = classify_comment_sentiment(entry["text"])
    comment_sentiments.append({"author": entry["author"], "comment": entry["text"], "sentiment": sentiment})

for item in comment_sentiments:
    print(f"Utente: {item['author']}")
    print(f"Commento: {item['comment']}")
    print(f"Sentiment: {item['sentiment']}")
    print("-" * 80)

## Risultato

La maggior parte dei commenti per il Submariner risulta positiva come mi aspettavo. 

# ANALISI ALTRO MARCHIO PER PARAGONE

Adesso voglio analizzare, per poi fare un paragone, un altro marchio di lusso dell'orologeria, ovvero **`Omega`**.
Quindi il mio obiettivo è estrarre i commenti, costruire un dataset per vedere la popolarità dei modelli, la community attorno al brand Omega e poi analizzare la similiartià nei comportamenti rispetto al brand **Rolex**.

In [ ]:
subreddits = ['watches','omegawatches']
omega_keywords = ['speedmaster', 'seamaster', 'aqua terra']
search_queries = [
    'Omega Speedmaster', 'Omega Seamaster', 'Omega Aqua Terra',
    'Omega Constellation', 'Speedy Tuesday', 'Moonwatch'
]
limit_per_query = 30000

data = []
seen_ids = set()

for sub in subreddits:
    print(f"Subreddit: r/{sub}")
    for query in search_queries:
        print(f"   → Query: {query}")
        for submission in reddit.subreddit(sub).search(query, sort="new", limit=limit_per_query):
            if submission.id in seen_ids:
                continue
            seen_ids.add(submission.id)

            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                if comment.body and comment.author:
                    text = comment.body.lower()
                    for model in omega_keywords:
                        if model in text:
                            data.append({
                                'author': comment.author.name,
                                'model': model,
                                'text': text,
                                'created_utc': pd.to_datetime(comment.created_utc, unit='s')
                            })
            time.sleep(1)

df = pd.DataFrame(data)
df.to_csv('reddit_comments_omega.csv', index=False)
print(f"Commenti salvati: {len(df)}")


In [ ]:
for d in data:
    d['created_utc'] = d['created_utc'].isoformat()

with open('reddit_comments_omega.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

with open('seen_ids_omega.json', 'w') as seen_file:
    json.dump(list(seen_ids), seen_file, indent=4)

print("Dizionari JSON salvati correttamente.")

 # Creo il grafo bipartito Omega

In [ ]:
with open('reddit_comments_omega.json', 'r') as f:
    data = json.load(f)

G_omega = nx.Graph()

edges = {(entry['author'], entry['model']) for entry in data if entry['author'] and entry['model']}

for author, model in edges:
    G_omega.add_node(author, bipartite=0)
    G_omega.add_node(model, bipartite=1)
    G_omega.add_edge(author, model)

nx.write_gexf(G_omega, "grafo_bipartito_omega.gexf")
print(f"Grafo bipartito Omega creato e salvato con {G_omega.number_of_nodes()} nodi e {G_omega.number_of_edges()} archi.")

![](bipartito_omega.png)

# Rete Bipartita: Utenti - Modelli Omega

Ho costruito una rete bipartita in cui:
- Un tipo di nodo rappresenta gli **utenti** Reddit che hanno commentato i modelli Omega.
- L'altro tipo rappresenta i **modelli Omega** ovvero Speedmaster, Seamaster ed Aqua Terra.
- Gli **archi** connettono ciascun utente ai modelli che ha commentato.

# Proiezione del grafo bipartito Omega sugli utenti

Per analizzare meglio la struttura della rete Omega, voglio proiettare il grafo bipartito sul set dei soli utenti. 

Così:
- Ogni nodo rappresenta un utente.
- Un arco esiste tra due utenti se hanno commentato almeno uno stesso modello Omega.
- Il peso degli archi può rappresentare la quantità di modelli commentati in comune.


In [ ]:
B = nx.read_gexf("grafo_bipartito_omega.gexf")

utenti = [n for n, d in B.nodes(data=True) if d["bipartite"] == 0]

G_omega_user = nx.bipartite.projected_graph(B, utenti)

nx.write_gexf(G_omega_user, "proiezione_utenti_omega.gexf")
print("Proiezione utenti Omega salvata in 'proiezione_utenti_omega.gexf'")

# Confronto tra la rete utenti Rolex e Omega

Ho proiettato i grafi bipartiti di entrambi i brand sui soli utenti, ottenendo due reti, così posso sia analizzare e confrontare la struttura delle community Reddit per ciascun marchio, che identificare quale rete risulta più attiva e connessa.

In [ ]:
def analizza_rete(G, nome_rete="Rete"):
    return {
        "Rete":nome_rete,
        "nodi": G.number_of_nodes(),
        "archi": G.number_of_edges(),
        "densità": nx.density(G),
        "grado_medio": sum(dict(G.degree()).values()) / G.number_of_nodes(),
        "componenti": nx.number_connected_components(G),
        "diametro": nx.diameter(G) if nx.is_connected(G) else "non connessa"
    }

analisi_rolex = analizza_rete(G_utenti, "Rolex")
analisi_omega = analizza_rete(G_omega_user, "Omega")

print(analisi_rolex)
print(analisi_omega)

La rete **Rolex** è più attiva e interconnessa. Le interazioni sono più frequenti e la distanza tra gli utenti è minore, quindi tra le due  community, sembrerebbe quella più coesa.

# Confronto utenti tra Rolex e Omega

Adesso voglio confrontare le community Reddit dei marchi Rolex e Omega analizzando i commenti degli utenti.  
Il mio obiettivo è capire se esistono utenti che commentano entrambi i brand.

Carico i due dizionari **utente --> modelli commentati** per ciascun marchio, poi faccio un'intersezione tra le chiavi dei due dizionari così mi restituisce gli utenti in comune.

In [ ]:
with open("reddit_comments_data.json", "r") as f:
    rolex_data = json.load(f)

with open("reddit_comments_omega.json", "r") as f:
    omega_data = json.load(f)

# Creo i dizionari utente → lista di modelli per entrambi i marchi di orologi
rolex_user_model_dict = {}
for entry in rolex_data:
    user = entry["author"]
    model = entry["model"]
    rolex_user_model_dict.setdefault(user, []).append(model)

omega_user_model_dict = {}
for entry in omega_data:
    user = entry["author"]
    model = entry["model"]
    omega_user_model_dict.setdefault(user, []).append(model)

In [ ]:
# Trovo gli utenti che hanno commentato entrambi i brand
common_users = set(rolex_user_model_dict.keys()) & set(omega_user_model_dict.keys())

print(f"Utenti che hanno commentato sia Rolex che Omega: {len(common_users)}")
for user in list(common_users)[:20]:  
    print(f"\nUtente: {user}")
    print(f"  Modelli Rolex: {set(rolex_user_model_dict[user])}")
    print(f"  Modelli Omega: {set(omega_user_model_dict[user])}")

# Grafo degli Utenti in Comune tra Rolex e Omega

Voglio costruire un grafo per rappresentare gli utenti che hanno commentato sia modelli Rolex che Omega, dove ogni utente è connesso ai modelli di entrambi i brand che ha commentato.

In [ ]:
G_duale = nx.Graph()

for user in common_users:
    G_duale.add_node(user, bipartite="user")

for user in common_users:
    for model in set(rolex_user_model_dict[user]):
        node_name = f"{model}"
        G_duale.add_node(node_name, bipartite="model", brand="Rolex")
        G_duale.add_edge(user, node_name)
        
for user in common_users:
    for model in set(omega_user_model_dict[user]):
        node_name = f"{model}"
        G_duale.add_node(node_name, bipartite="model", brand="Omega")
        G_duale.add_edge(user, node_name)

nx.write_gexf(G_duale, "grafo_utenti_comuni_rolex_omega.gexf")
print("Grafo creato: grafo_utenti_comuni_rolex_omega.gexf")

![](rolex_omega.png)

# Confronto tra il Grafo Bipartito e la Proiezione sugli Utenti

Nel mio progetto ho costruito due tipologie di grafo per analizzare le dinamiche tra utenti Reddit e i modelli di orologi dei brand Rolex e Omega:

### 1. Grafo Bipartito (Utenti --> Modelli)

- **Nodi**: utenti + modelli (es. *Submariner*, *Speedmaster*, ecc.)
- **Archi**: collegano un utente a un modello che ha commentato.
  - Capire quali modelli attirano più attenzione.
  - Analizzare la distribuzione degli utenti attorno ai diversi brand.
  - Visualizzare le connessioni tra utenti e prodotti (es. utenti "ibridi").

### 2. Proiezione sulla Rete degli Utenti

- **Nodi**: solo utenti
- **Archi**: due utenti sono collegati se hanno commentato almeno un modello in comune (anche tra brand diversi).
  - Identificare comunità coese (community detection).
  - Confrontare le strutture delle reti Rolex vs Omega.

In [ ]:
with open("reddit_comments_data.json", "r") as f:
    rolex_data = json.load(f)
with open("reddit_comments_omega.json", "r") as f:
    omega_data = json.load(f)

rolex_dict = {}
for entry in rolex_data:
    rolex_dict.setdefault(entry["author"], set()).add(entry["model"])

omega_dict = {}
for entry in omega_data:
    omega_dict.setdefault(entry["author"], set()).add(entry["model"])

# Unisco gli  utenti
all_users = set(rolex_dict) | set(omega_dict)

brand_activity = {}
for user in all_users:
    rolex = user in rolex_dict
    omega = user in omega_dict
    if rolex and omega:
        brand_activity[user] = "Entrambi"
    elif rolex:
        brand_activity[user] = "Rolex"
    else:
        brand_activity[user] = "Omega"

G_pro_unito = nx.Graph()

for user in all_users:
    G_pro_unito.add_node(user, brand=brand_activity[user])

user_list = list(all_users)
for i in range(len(user_list)):
    for j in range(i + 1, len(user_list)):
        u1 = user_list[i]
        u2 = user_list[j]

        modelli_u1 = rolex_dict.get(u1, set()) | omega_dict.get(u1, set())
        modelli_u2 = rolex_dict.get(u2, set()) | omega_dict.get(u2, set())

        if modelli_u1 & modelli_u2:
            G_pro_unito.add_edge(u1, u2)

nx.write_gexf(G_pro_unito, "proiezione_utenti_rolex_omega.gexf")
print("Proiezione utenti unificata salvata in 'proiezione_utenti_rolex_omega.gexf'")

![](utenti_rolex_omega.png)

- **Verde:** Utenti che commentano esclusivamente modelli Rolex.
- **Rosso:** Utenti che commentano esclusivamente modelli Omega.
- **Azzurro:** Utenti che commentano entrambi i brand.

Ogni nodo rappresenta un utente e un link tra due nodi indica che hanno commentato almeno un modello in comune.

In [ ]:
degree_dict = dict(G_pro_unito.degree())

betweenness = nx.betweenness_centrality(G_pro_unito)

data = []
for node, attr in G_pro_unito.nodes(data=True):
    brand = attr.get("brand")
    data.append({
        "user": node,
        "brand": brand,
        "degree": degree_dict.get(node, 0),
        "betweenness": betweenness.get(node, 0)
    })

df_metrics = pd.DataFrame(data)

summary = df_metrics.groupby("brand")[["degree", "betweenness"]].mean().round(3)
print(summary)

# PREDIRE QUALE BRAND TRA ROLEX E OMEGA LA GENTE PREFERISCE

Il mio obiettivo è **predire il brand preferito** di un utente. 

Quindi vorrei capire grazie ad un modello di **Machine Learning** se dato il comportamento testuale di un utente, riesco a predire se preferisce Rolex o Omega.


In [ ]:
with open("reddit_comments_data.json", "r") as f:
    rolex_data = json.load(f)

with open("reddit_comments_omega.json", "r") as f:
    omega_data = json.load(f)

def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

rolex_users = {}
for entry in rolex_data:
    user = entry["author"]
    text = entry.get("text", "")
    if user not in rolex_users:
        rolex_users[user] = {"brand": "Rolex", "n_commenti": 0, "total_sentiment": 0}
    rolex_users[user]["n_commenti"] += 1
    rolex_users[user]["total_sentiment"] += get_sentiment(text)

omega_users = {}
for entry in omega_data:
    user = entry["author"]
    text = entry.get("text", "")
    if user not in omega_users:
        omega_users[user] = {"brand": "Omega", "n_commenti": 0, "total_sentiment": 0}
    omega_users[user]["n_commenti"] += 1
    omega_users[user]["total_sentiment"] += get_sentiment(text)

# Unisco i dizionari (ma solo utenti che hanno commentato un solo brand)
all_users = {}
for user, info in rolex_users.items():
    if user not in omega_users:
        all_users[user] = info
for user, info in omega_users.items():
    if user not in rolex_users:
        all_users[user] = info

df = pd.DataFrame.from_dict(all_users, orient="index")
df["avg_sentiment"] = df["total_sentiment"] / df["n_commenti"]
df = df[["n_commenti", "avg_sentiment", "brand"]]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df["brand_label"] = df["brand"].map({"Rolex": 1, "Omega": 0})

X = df[["n_commenti", "avg_sentiment"]]
y = df["brand_label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Modello
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["Omega", "Rolex"]))

# Valutazione del Modello di Classificazione

Per valutare il modello di Machine Learning che predice se un utente preferisce Rolex o Omega, ho utilizzato:

- **Accuracy**: percentuale di previsioni corrette, che corrisponde a 0.52, qui  il modello predice correttamente il brand in circa il 52% dei casi.
- **Classification Report**: include **precision, recall e F1-score** per ciascuna classe.
- **Confusion Matrix**: mostra il numero di predizioni corrette e sbagliate per ciascuna categoria, dove ho scelto **Rolex** come classe positiva.

## Metriche:
- **Precision:** mi indica quanti dei commenti classificati come appartenenti a un brand sono realmente corretti. Qui Rolex ha una precisione maggiore, 0,57, ma commette più errori in recall.
- **Recall:** mi misura la capacità del modello di identificare tutti i commenti realmente appartenenti a un brand. Qui Omega ottiene un valore più alto ovvero 0.62.
- **F1-score:** mi da la  media armonica tra precision e recall. Valore bilanciato leggermente migliore per Omega.
- **Support:** è il numero di esempi reali per ciascuna classe.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calcolo la confusion matrix con ordine: [Omega, Rolex]
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])  # 0 = Omega, 1 = Rolex

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Omega", "Rolex"],
            yticklabels=["Omega", "Rolex"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Rolex vs Omega")
plt.tight_layout()  
plt.show()

In questa mia matrice di confusione:
- **True Positives** (TP): 305 per Rolex.
- **True Negatives** (TN): 390 per Omega.
- **False Positives** (FP): 234 utenti che in realtà preferivano Omega sono stati erroneamente classificati come Rolex, quindi il modello sovrastima Rolex. 
- **False Negatives** (FN): 402 utenti che preferivano Rolex sono stati erroneamente classificati come Omega.

Questo modello tende quindi a sottostimare Rolex visto che commette più errori nel riconoscere Rolex, con molti utenti che preferivano Rolex classificati come Omega, pertanto **Omega viene classificato meglio di Rolex**.

# Curva ROC e AUC

Per valutare la capacità del modello di distinguere tra i due brand voglio utilizzare la **ROC Curve (Receiver Operating Characteristic)** e il relativo **AUC (Area Under the Curve)**.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_prob = clf.predict_proba(X_test)[:, 1] 

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
auc = roc_auc_score(y_test, y_prob)  
print(f"AUC: {auc}")

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot()
ax.plot(fpr, tpr, linewidth=2, label=f"ROC curve (AUC = {auc:.2f})")
ax.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
ax.set_xlabel('False Positive Rate - FPR')
ax.set_ylabel('True Positive Rate (Recall)')
ax.axis([0, 1, 0, 1])
ax.legend(loc="lower right", fontsize=13)
plt.title("ROC Curve")
plt.grid(True)
plt.show()

Il mio modello ha una capacità moderata di distinguere tra gli utenti che preferiscono Rolex e quelli che preferiscono Omega.
- Un **AUC di 0.53** significa che, dato un esempio positivo, Rolex, e uno negativo, Omega, c’è il 53% di probabilità che il modello assegni un punteggio più alto al positivo. 
Quindi come già visto nella matrice di confusione, il modello fatica a separare bene le due classi.
- Non è ottimo, perchè è leggermente migliore al caso casuale AUC = 0.50.

# Predizione della Preferenza tra Rolex e Omega tramite Logistic Regression

Il mio obiettivo adesso è predire quale brand di orologi un utente preferisce tra Rolex e Omega, sfruttando però le caratteristiche strutturali della rete utente. Andrò ad applicare un algoritmo di classificazione, quello di **Logistic Regression**.

Utilizzo come variabili indipendenti sia caratteristiche testuali (numero di commenti e sentiment medio), sia caratteristiche strutturali della rete (degree e betweenness centrality), con l’obiettivo di migliorare la capacità predittiva del modello.

Nel mio modello di regressione logistica,andrò ad assegnare il valore 1 a Rolex e 0 a Omega. Questo significa che la regressione predice la probabilità che un utente preferisca Rolex.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix


df["brand_label"] = df["brand"].map({"Omega": 0, "Rolex": 1})

X = df[["n_commenti", "avg_sentiment", "degree", "betweenness"]]
y = df["brand_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

ct = ColumnTransformer([
    ('minmax', MinMaxScaler(), ["n_commenti", "avg_sentiment", "degree", "betweenness"])
])

model = Pipeline([
    ('preprocessing', ct),
    ('classifier', LogisticRegression())
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy:  {acc:}")
print(f"Recall:    {recall}")
print(f"Precision: {precision}")
print(f"F1:        {f1}")
print("\nConfusion Matrix:")
print(cm)

coefs = model['classifier'].coef_
print(f"\n{coefs}")

## Valutazione del Modello

Il modello ha ottenuto una **accuracy del 63%**, con uno **F1 pari a circa 65%**. Questo indica che la regressione logistica è in grado di discriminare, in modo moderato, tra utenti che preferiscono Rolex e quelli che preferiscono Omega.

La **matrice di confusione** mostra il numero di predizioni corrette e incorrette per ciascuna classe e mostra che il modello riesce a identificare meglio gli utenti che preferiscono Rolex (469) rispetto a quelli che preferiscono Omega (367). Inoltre, i falsi positivi, Omega predetto come Rolex, sono più alti dei falsi negativi.

Dall'analisi dei **coefficienti del modello**:
- Coefficienti positivi: spingono verso la classe positiva (Rolex)
- Coefficienti negativi: spingono verso la classe negativa (Omega)

Inoltre
- **n_commenti** ha un impatto positivo, indicando che più commenti possono indicare una preferenza per Rolex.
- **betweenness** ha un impatto negativo marcato, suggerendo che utenti centrali nella rete preferiscono tendenzialmente Omega.

Il modello **predice Rolex** quando:
- l’utente ha molti commenti
- ha un alto degree 

Il modello **predice Omega** quando:
- l’utente ha sentiment molto positivo
- ha alta betweenness 

### Analisi Confronto Omega vs Role: Preferenze e Coinvolgimento

Identifico nuovamennte gli utenti che hanno commentato sia Rolex che Omega con degli esempi.

In [ ]:
rolex_users = set([entry["author"] for entry in rolex_data])
omega_users = set([entry["author"] for entry in omega_data])
common_users = rolex_users.intersection(omega_users)

print(f"Utenti che hanno commentato entrambi i brand: {len(common_users)}")
print("Esempi:", list(common_users)[:10])

### Distribuzione utenti tra Rolex e Omega

Vedo quale brand ha attratto più utenti unici.

In [ ]:
print(f"Utenti unici Rolex: {len(rolex_users)}")
print(f"Utenti unici Omega: {len(omega_users)}")

### Commenti per modello

Analisi su quale modello è più discusso tra i brand.

In [ ]:
from collections import Counter

rolex_models = Counter([entry["model"] for entry in rolex_data])
omega_models = Counter([entry["model"] for entry in omega_data])

print("Modelli Rolex più commentati:", rolex_models.most_common(3))
print("Modelli Omega più commentati:", omega_models.most_common(3))

In [ ]:
with open("reddit_comments_data.json", "r") as f:
    rolex_data = json.load(f)
with open("reddit_comments_omega.json", "r") as f:
    omega_data = json.load(f)

user_model_map = defaultdict(set)

# Inserisco i commenti Rolex
for entry in rolex_data:
    user_model_map[entry["author"].lower()].add(entry["model"].lower())

# Inserisco i commenti Omega
for entry in omega_data:
    user_model_map[entry["author"].lower()].add(entry["model"].lower())

modelli_target = ["gmt", "submariner", "datejust", "speedmaster", "seamaster", "aqua terra"]

df = pd.DataFrame([
    {model: int(model in models) for model in modelli_target}
    for user, models in user_model_map.items()
], index=list(user_model_map.keys()))

def infer_brand_preference(row):
    rolex = row[["gmt", "submariner", "datejust"]].sum()
    omega = row[["speedmaster", "seamaster", "aqua terra"]].sum()
    if rolex > omega:
        return "Rolex"
    elif omega > rolex:
        return "Omega"
    else:
        return "Equal"

df["brand_preference"] = df.apply(infer_brand_preference, axis=1)

df.head(20)

In [ ]:
percentuali = df["brand_preference"].value_counts(normalize=True) * 100
print(percentuali.round(2).to_dict())

# Sentiment 

Infine adesso brevemente analizzo il tono dei commenti, positivo, neutro oppure negativo, per ciascun brand così da capire quale brand riceve sentiment più favorevole da parte degli utenti.

In [ ]:
# Unisco tutti i commenti in una lista unica con brand
commenti = []

for entry in rolex_data:
    commenti.append({"text": entry["text"], "brand": "Rolex"})

for entry in omega_data:
    commenti.append({"text": entry["text"], "brand": "Omega"})

def analizza_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return "Positivo"
    elif polarity < 0:
        return "Negativo"
    else:
        return "Neutro"

for commento in commenti:
    commento["sentiment"] = analizza_sentiment(commento["text"])

df_sentiment = pd.DataFrame(commenti)

report = df_sentiment.groupby("brand")["sentiment"].value_counts(normalize=True).unstack().round(2) * 100
print(report)

In [ ]:
sentiment_counts = {
    'Rolex': {'Positivo': 68, 'Neutro': 17, 'Negativo': 15},
    'Omega': {'Positivo': 68, 'Neutro': 21, 'Negativo': 11}
}

df_sentiment = pd.DataFrame(sentiment_counts).T

df_percentuali = df_sentiment.div(df_sentiment.sum(axis=1), axis=0) * 100

ax = df_percentuali.plot(kind='bar', stacked=True, figsize=(8, 5), color=["green", "gray", "red"])

plt.title("Distribuzione del Sentiment per Brand")
plt.ylabel("Percentuale (%)")
plt.xlabel("Brand")
plt.legend(title="Sentiment")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Come mi aspettvo ho rilevato un sentiment molto simile verso Rolex e Omega, infatti i commenti sono prevalentemente positivi. C'è una percentuale leggermente più grande per i "neutri" per quanto riguarda Omega.

Comunque, ho una percezione positiva e consolidata di entrambi i marchi nelle community analizzate.

# Metodi di Ensemble per la Predizione della Preferenza tra Rolex e Omega

In questa sezione, ho voluto applicare **algoritmi di Ensemble Learning** per migliorare la capacità predittiva del mio modello, sfruttando combinazioni di più classificatori deboli.

In [ ]:
X = df[["n_commenti", "avg_sentiment"]]
y = df["brand_label"]  

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

## Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

clf1 = LogisticRegression()
clf2 = DecisionTreeClassifier()
clf3 = RandomForestClassifier()

voting_clf = VotingClassifier(estimators=[
    ('lr', clf1), ('dt', clf2), ('rf', clf3)
], voting='hard')

voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)
print("Accuracy Voting:", accuracy_score(y_test, y_pred_voting))

## Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    random_state=42
)

bagging_clf.fit(X_train, y_train)
y_pred_bagging = bagging_clf.predict(X_test)
print("Accuracy Bagging:", accuracy_score(y_test, y_pred_bagging))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Accuracy Random Forest:", accuracy_score(y_test, y_pred_rf))

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

adaboost_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=50,
    learning_rate=1,
    random_state=42
)

adaboost_clf.fit(X_train, y_train)
y_pred_adaboost = adaboost_clf.predict(X_test)
print("Accuracy AdaBoost:", accuracy_score(y_test, y_pred_adaboost))

## Confronto tra modelli 

In [ ]:
models = ['Voting', 'Bagging', 'Random Forest', 'AdaBoost']
accuracies = [
    accuracy_score(y_test, y_pred_voting),
    accuracy_score(y_test, y_pred_bagging),
    accuracy_score(y_test, y_pred_rf),
    accuracy_score(y_test, y_pred_adaboost)
]

colors = ['skyblue', 'lightgreen', 'salmon', 'plum']

plt.figure(figsize=(8, 6))
bars = plt.bar(models, accuracies, color=colors)

for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title("Confronto modelli Ensemble per predire preferenza tra Rolex e Omega")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Metodi di Ensemble per la Predizione 
## Predire la Preferenza tra Rolex e Omega

L’obiettivo è predire, a partire da informazioni testuali e strutturali degli utenti su Reddit, se un utente preferisce **Rolex** o **Omega**. Utilizzeremo tre modelli:

- **Decision Tree** (modello semplice, interpretabile)
- **Random Forest** (ensemble di alberi, più robusto)
- **Bagging** (ensemble generico con Decision Tree)

Useremo come feature:
- Numero di commenti
- Sentiment medio

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42)
tree_clf.fit(X_train, y_train)

y_train_pred = tree_clf.predict(X_train)
y_test_pred = tree_clf.predict(X_test)

acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)

print(f"Decision Tree train/test accuracy: {acc_train:.3f} / {acc_test:.3f}")

L’accuratezza bassa sia in train che in test indica che **l’albero di decisione non riesce a catturare pattern significativi dai dati**.
- il modello **non è overfittato**, ma è debole: probabilmente i dati non contengono sufficiente informazione predittiva oppure sono troppo rumorosi.
- Performance vicina al random guessing, ovvero 0.5 quindi non è utile per la predizione.

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_clf.fit(X_train, y_train)

y_train_pred_rf = rf_clf.predict(X_train)
y_test_pred_rf = rf_clf.predict(X_test)

acc_train_rf = accuracy_score(y_train, y_train_pred_rf)
acc_test_rf = accuracy_score(y_test, y_test_pred_rf)

print(f"Random Forest train/test accuracy: {acc_train_rf:.3f} / {acc_test_rf:.3f}")

Qui ho un’**accuratezza migliore**, segno che riesce a cogliere un po’ di struttura nei dati.
- Il test scende a 0.547, solo leggermente sopra il caso casuale.
- L’AUC visto prima era circa 0.53, coerente con questi numeri: il modello riesce a fare un po’ meglio del caso, ma non abbastanza da poter dire che abbia potere predittivo reale.
- La **prestazione migliora rispetto all’albero** singolo ma ancora non affidabile per la classificazione.

## Bagging Classifier con Decision Tree

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=5),
    n_estimators=100,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

bagging_clf.fit(X_train, y_train)

y_train_pred_bag = bagging_clf.predict(X_train)
y_test_pred_bag = bagging_clf.predict(X_test)

acc_train_bag = accuracy_score(y_train, y_train_pred_bag)
acc_test_bag = accuracy_score(y_test, y_test_pred_bag)

print(f"Bagging train/test accuracy: {acc_train_bag:.3f} / {acc_test_bag:.3f}")

**Accuracy quasi uguale a quella del decision tree singolo**.
- Bagging aiuta a ridurre la varianza, ma in questo caso l’instabilità del decision tree base non è sufficiente per generare un guadagno evidente. 
- Risultato molto vicino al caso casuale, quindi anche qui poco utile.

In [ ]:
models = ['Decision Tree', 'Random Forest', 'Bagging']

# Accuracy dei test set,
accuracies = [0.533, 0.547, 0.538]

colors = ['skyblue', 'lightgreen', 'salmon']

plt.figure(figsize=(8, 6))
bars = plt.bar(models, accuracies, color=colors)

for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title("Confronto modelli di classificazione\nper predire preferenza tra Rolex e Omega")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

### **Nessuno dei modelli che ho testato ha purtroppo raggiunto un livello di accuratezza che dimostri una reale capacità predittiva**. 
Confrontando le accurac dei 3 modelli, anche se **Random Forest** ha una lievissima superiorità ,0.547, nessuno supera abbondantemente il caso casuale di 0.50
- I dati disponibili non contengono abbastanza segnale predittivo per distinguere in modo affidabile chi preferisce Rolex o Omega.
- Reddit ospita una community di appassionati di orologi, ma le preferenze espresse nei commenti non sono così nette da poter essere codificate da un modello supervisionato.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt

X = df[["n_commenti", "avg_sentiment"]]
y = df["brand_label"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=42)

tree_clf = DecisionTreeClassifier(max_depth=5, random_state=1)
bag = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=5), n_estimators=100, random_state=1)
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=1, max_features=1)

fig = plt.figure(figsize=(12, 4))
labels = ['Decision Tree', 'Bagging', 'Random Forest']

for index, (clf, lab) in enumerate(zip([tree_clf, bag, rf_clf], labels)):
    clf.fit(X_train, y_train)
    f1_train = f1_score(y_train, clf.predict(X_train))
    f1_test = f1_score(y_test, clf.predict(X_test))


    ax = fig.add_subplot(1, 3, index + 1)
    ax.set_title(f"{lab}\n train/test F1: {f1_train:.3f}/{f1_test:.3f}")
    plot_decision_regions(X=X_train, y=y_train.values, clf=clf, ax=ax, legend=2)

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 4))
labels = ['Decision Tree', 'Bagging', 'Random Forest']

for index, (clf, lab) in enumerate(zip([tree_clf, bag, rf_clf], labels)):
    clf.fit(X_train, y_train)
    f1_train = f1_score(y_train, clf.predict(X_train))
    f1_test = f1_score(y_test, clf.predict(X_test))
    
    ax = fig.add_subplot(1, 3, index + 1)
    ax.set_title(f'{lab}\n train/test F1: {f1_train:.3f}/{f1_test:.3f}')
    
    plot_decision_regions(X=X_train, y=y_train.values, clf=clf, ax=ax, legend=2)

 # Limitato con itervvallo sull'asse X
    ax.set_xlim(0, 6)

plt.tight_layout()
plt.show()

Tre modelli di classificazione:
- **Decision Tree**
- **Bagging Classifier**
- **Random Forest**

Ho utilizzato come variabili esplicative:
- **n_commenti**: numero di commenti lasciati dall’utente
- **avg_sentiment**: sentimento medio associato ai commenti (standardizzato)
- I dati li ho **normalizzati** con **StandardScaler()**
- Ho **limitato l’intervallo degli assi X** a [0, 6] per evitare compressione visiva

Risultati (F1 Score Train/Test):
- **Decision Tree:** 0.636 / 0.608  
- **Bagging:** 0.646 / 0.613  
- **Random Forest:** 0.654 / 0.619  

Il grafico mostra le *regioni di decisione* create da ciascun classificatore. Le zone blu rappresentano la classe 0 (Omega), quelle arancioni la classe 1 (Rolex).

- **Decision Tree** tende ad overfittare leggermente, con confini più rigidi e netti.
- **Bagging** migliora la stabilità, rendendo le regioni più coerenti.
- **Random Forest** produce regioni frammentate, ma con una maggiore capacità predittiva sui dati test.

L’uso di metodi ensemble come Bagging e Random Forest aumenta di poco l’accuratezza predittiva sulla preferenza tra Rolex e Omega rispetto a un singolo albero decisionale, però non è sufficiente.